In [2]:
import cv2

In [ ]:
from ultralytics import YOLO
model = YOLO("yolo-Weights/yolov8n.pt")

100%|██████████| 6.25M/6.25M [00:00<00:00, 7.16MB/s]


In [53]:
import cv2
import math
from ultralytics import YOLO
import time  # For cooldown timer

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

model = YOLO("yolo-Weights/yolov8n.pt", verbose=False)

classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"]

# Variables to control cooldown and tracking
target_class = "cell phone"  # Object class to detect
cooldown_time = 5  # Seconds to wait before triggering the action again
last_action_time = 0  # Track the last time an action was executed
threshold = 0.5

while True:
    success, img = cap.read()
    if not success:
        break

    results = model(img, stream=True, verbose=False)
    objects_detected = []  # Store positions of objects of the target class

    for r in results:
        boxes = r.boxes

        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Calculate center coordinates
            x_centre = (x1 + x2) // 2
            y_centre = (y1 + y2) // 2

            confidence = float(box.conf[0])
            cls = int(box.cls[0])
            detected_class = classNames[cls]

            # Only proceed if confidence is above the threshold
            if confidence >= threshold:
                # Draw bounding box and center dot
                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
                cv2.circle(img, (x_centre, y_centre), radius=5, color=(0, 0, 255), thickness=-1)
                cv2.putText(img, f"{detected_class} {confidence:.2f}", (x1, y1 - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

                # If the detected class matches the target, add to the list
                if detected_class == target_class:
                    objects_detected.append((x_centre, y_centre))

    # Check if any objects of the target class were detected
    if objects_detected:
        # Only execute the action if cooldown time has passed
        current_time = time.time()
        if current_time - last_action_time > cooldown_time:
            print(f"Detected {len(objects_detected)} {target_class}(s): {objects_detected}")
            # Perform your custom action here
            for idx, (x, y) in enumerate(objects_detected):

                ## detect whether object in left or right of image:
                ## take left as 0 and right as 1
                side_of_image = x > 320 # half image size
                print(f"Object {idx + 1} - Center: ({x}, {y})")
                print(f'Obect is in right of image: {side_of_image}')
            # Update last action time
            last_action_time = current_time

    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Detected 1 cell phone(s): [(157, 304)]
Object 1 - Center: (157, 304)
Obect is in right of image: False
Detected 1 cell phone(s): [(274, 312)]
Object 1 - Center: (274, 312)
Obect is in right of image: False
Detected 2 cell phone(s): [(187, 321), (477, 311)]
Object 1 - Center: (187, 321)
Obect is in right of image: False
Object 2 - Center: (477, 311)
Obect is in right of image: True
Detected 2 cell phone(s): [(449, 293), (184, 311)]
Object 1 - Center: (449, 293)
Obect is in right of image: True
Object 2 - Center: (184, 311)
Obect is in right of image: False
Detected 2 cell phone(s): [(473, 287), (183, 305)]
Object 1 - Center: (473, 287)
Obect is in right of image: True
Object 2 - Center: (183, 305)
Obect is in right of image: False
